In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [3]:
cd ./pointnet.pytorch

/content/drive/MyDrive/pointnet.pytorch


In [4]:
!pip install plyfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Convert

In [ ]:
import os
import glob
import multiprocessing
from multiprocessing import Pool
from pathlib import Path
from plyfile import PlyData
from tqdm import tqdm, trange
import time
import pdb
import gc

def preprocess_off(in_file):

    with open(in_file,"rt") as f:
        # some OFF files in original dataset had OFF345 345 344 where 
        # OFF collided with the number. Needs \n
        lines = f.readlines()
    if lines[0] != 'OFF\n':
        with open(in_file,"wt") as f:
            lines[0] = lines[0][0:3] + '\n' + lines[0][3:]
            lines = "".join(lines)
            f.write(lines)
            f.close()
    else:
        f.close()

def offFormat_to_plyFormat(C_file):

    with open(C_file,"rt") as Cf:
        lines = Cf.readlines()
    with open(C_file,"wt") as Cf:
        num_points = lines[1].split()[0]
        num_faces = lines[1].split()[1]
        lines[0] = 'ply\n'
        lines[1] = 'format ascii 1.0\n'+\
                    'element vertex %s'%num_points+'\n'+\
                    'property float x\n'+\
                    'property float y\n'+\
                    'property float z\n'+\
                    'element face %s'%num_faces+'\n'+\
                    'property list uchar int vertex_index\n'+\
                    'end_header\n'
        lines = "".join(lines)
        Cf.write(lines)
        Cf.close()


def full_off_to_ply(p, l, chunksize, num_cpu=None):

    print('searching path', p, 'to convert .off files to .ply')

    if (num_cpu != None):
        start = time.time()
        with Pool(processes = num_cpu) as po0:
            for i in po0.imap_unordered(preprocess_off, l, chunksize=chunksize):
                pass
        with Pool(processes = num_cpu) as po1:
            for i in po1.imap_unordered(offFormat_to_plyFormat, l, chunksize=chunksize):
                pass
        end = time.time()
        print("Muti core full_off_to_ply use %3f sec!" %(end - start))

    else:
        start = time.time()
        for f in l:
            preprocess_off(f)
        for f in l:
            offFormat_to_plyFormat(f)
        end = time.time()
        print("Single core full_off_to_ply use %3f sec!" %(end - start))




def process_txt():
    # Convert test.txt, train.txt, trainval.txt, val.txt data .off to .ply.
    txtNames = glob.glob(r"*.txt")

    for txtName in txtNames:
        with open(txtName,"rt") as file:
            x = file.read()
        with open(txtName, "wt") as file:
            x = x.replace(".off",".ply")
            file.write(x)

def suffix(l):
    # Convert all .off suffix to .ply.
    for i in l:
        tmp = i.with_suffix('.ply')
        os.rename(i, tmp)


def sub_check(f):

    plydata = PlyData.read(f)
    del plydata
    gc.collect()
    

def check(p, chunksize, num_cpu=None):
    # Use PlyData.read(f) check .ply, if ply file broken, sub_check will crush.
    ply = list(p.glob('**/*.ply'))
    progress = tqdm(total=len(ply))

    if (num_cpu != None):
        start = time.time()
        with Pool(processes = num_cpu) as po:
            for i in po.imap_unordered(sub_check, ply, chunksize=chunksize):
                progress.update(1)
        end = time.time()
        print("Muti core check use %3f sec!" %(end - start))
    else:
        start = time.time()
        for f in ply:
            sub_check(f)
            progress.update(1)
        end = time.time()
        print("Single core check use %3f sec!" %(end - start))
num_cpu = multiprocessing.cpu_count()
chunksize = 100
currPATH = os.getcwd().replace('\\','/')
p = Path(currPATH)
l = list(p.glob('**/*.off'))
if isinstance(num_cpu, int):
    print("Now use %d Threads!" %num_cpu)
else:
    num_cpu = None
    print("Now use single Thread!")
full_off_to_ply(p, l, chunksize, num_cpu)
process_txt()
suffix(l)
check(p, chunksize, num_cpu)
print('ur great!')
print('All .off format has converted!')

Now use 2 Threads!
searching path /content/drive/MyDrive/pointnet.pytorch to convert .off files to .ply
Muti core full_off_to_ply use 3234.549246 sec!


100%|██████████| 12311/12311 [1:56:04<00:00,  1.77it/s]

Muti core check use 6964.572830 sec!
ur great!
All .off format has converted!


In [ ]:
!python train_classification.py --dataset ./data/ModelNet40 --nepoch=1 --dataset_type modelnet40

# Code

In [6]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm
from plyfile import PlyData
import numpy as np

BATCHSIZE = 32
NPOINTS = 2500
EPOCH = 5
WORKERS = 2
DATAPATH = "./data/ModelNet40"
device = torch.device('cuda') if torch.cuda.is_available() is True else torch.device('cpu')

# Dataset

In [7]:
class ModelNetDataset(Dataset):
  def __init__(self, dataPath, n_points=2500, mode='train', augmentation=True):
    with open('./misc/modelnet_id.txt') as f:
      ids = [n.strip().split() for n in f.readlines()]
    category_to_id = {}
    for cat, i in ids:
      category_to_id[cat] = int(i)
    with open(os.path.join(dataPath, f'{mode}.txt'), 'r') as f:
      fileNames = [n.strip() for n in f.readlines()]
    self.files = [(os.path.join(dataPath, n), category_to_id[n.split('/')[0]]) for n in fileNames] # (path, category)
    self.n_points = n_points
    self.augmentation = augmentation
    self.category = len(category_to_id.keys())
    self.device = device

  def __len__(self):
    return len(self.files)

  def __getitem__(self, index):
    path, category = self.files[index] # eg. glass_box/train/glass_box_0090.ply
    points = self.read_points(path) # ex. (11634, 3)
    points = self.sampling(points, self.n_points) # (n_points, 3)
    points = self.standardization(points)
    if self.augmentation is True:
      points = self.augment_points(points)
    return torch.from_numpy(points.astype(np.float32)), torch.from_numpy(np.array([category]).astype(np.int64))

  def read_points(self, path):
    with open(path, 'rb') as f:
        plydata = PlyData.read(f)
    p_x, p_y, p_z = plydata['vertex']['x'], plydata['vertex']['y'], plydata['vertex']['z']
    points = np.vstack((p_x, p_y, p_z)).T # ex. (11634, 3)
    return points
  
  def sampling(self, points, n_points):
    choice = np.random.choice(len(points), n_points, replace=True)
    return points[choice, :]
  
  def standardization(self, points):
    epsilon = 1e-7
    return (points - np.mean(points, axis=0)) / (np.std(points, axis=0) + epsilon)
  
  def augment_points(self, points):
    self.rotation(points[:, [0, 2]])
    self.jitter(points)
    return points
  
  def rotation(self, matrix):
    theta = np.random.uniform(0, np.pi * 2)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    matrix[:] = matrix.dot(rotation_matrix)
  
  def jitter(self, points):
    points[:] = points + np.random.normal(0, 0.02, points.shape)

# Model

In [8]:
class STN(nn.Module):
  """
  Spatial Transformer Network
  """
  def __init__(self, dim):
    super(STN, self).__init__()
    self.dim = dim
    self.conv1 = torch.nn.Conv1d(dim, 64, 1)
    self.conv2 = torch.nn.Conv1d(64, 128, 1)
    self.conv3 = torch.nn.Conv1d(128, 1024, 1)
    self.fc1 = nn.Linear(1024, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, dim*dim)
    self.relu = nn.ReLU()
    self.bn1 = nn.BatchNorm1d(64)
    self.bn2 = nn.BatchNorm1d(128)
    self.bn3 = nn.BatchNorm1d(1024)
    self.bn4 = nn.BatchNorm1d(512)
    self.bn5 = nn.BatchNorm1d(256)

  def forward(self, x): # (batch, dim, n_points)
    batch, dim, n_points = x.shape # ex. batch=32, n_points=2500
    x = self.relu(self.bn1(self.conv1(x))) # (32, 64, 2500)
    x = self.relu(self.bn2(self.conv2(x))) # (32, 128, 2500)
    x = self.relu(self.bn3(self.conv3(x))) # (32, 1024, 2500)
    x, _ = torch.max(x, dim=2) # pointwise pooling (32, 1024)
    x = self.relu(self.bn4(self.fc1(x))) # (32, 512)
    x = self.relu(self.bn5(self.fc2(x))) # (32, 256)
    x = self.fc3(x) # (32, dim*dim)
    iden = torch.eye(dim).view(1, -1)
    iden = iden.cuda() if x.is_cuda else iden
    x = x + iden
    x = x.view(-1, dim, dim) # (32, dim, dim)
    return x


class PointNetFeature(nn.Module):
  def __init__(self): 
    super(PointNetFeature, self).__init__()
    self.stn = STN(3)
    self.fstn = STN(64)
    self.conv1 = torch.nn.Conv1d(3, 64, 1)
    self.conv2 = torch.nn.Conv1d(64, 128, 1)
    self.conv3 = torch.nn.Conv1d(128, 1024, 1)
    self.bn1 = nn.BatchNorm1d(64)
    self.bn2 = nn.BatchNorm1d(128)
    self.bn3 = nn.BatchNorm1d(1024)
    self.relu = nn.ReLU()
  
  def forward(self, x): # (batch, 3, n_points)
    x = torch.bmm(x.transpose(1, 2), self.stn(x)) # (batch, n_points, 3)
    x = x.transpose(1, 2) # (batch, 3, n_points)
    x = self.relu(self.bn1(self.conv1(x))) # (batch, 64, n_points)
    trans_mat = self.fstn(x) # (32, 64, 64)
    x = x.transpose(1, 2)
    x = torch.bmm(x, trans_mat) # (batch, n_points, 64)
    x = x.transpose(1, 2) # (batch, 64, n_points)
    local_feature = x
    x = self.relu(self.bn2(self.conv2(x)))  # (batch, 128, n_points)
    x = self.relu(self.bn3(self.conv3(x)))  # (batch, 1024, n_points)
    global_feature, _ = torch.max(x, dim=-1)
    return global_feature, local_feature, trans_mat # (b, 1024), (b, 64, n), (b, 64, 64), trans_mat is returned together for regularization

class PointNet(nn.Module):
  def __init__(self, category=2, segmentation=False):
    super(PointNet, self).__init__()
    self.feat = PointNetFeature()
    if segmentation is False:
      self.linear1 = nn.Linear(1024, 512) # classification
      self.linear2 = nn.Linear(512, 256)
      self.linear3 = nn.Linear(256, category) # classification
      self.bn1 = nn.BatchNorm1d(512)
      self.bn2 = nn.BatchNorm1d(256)
      self.dropout = nn.Dropout(p=0.3)
    else:
      self.conv1 = torch.nn.Conv1d(1024+64, 512, 1)
      self.conv2 = torch.nn.Conv1d(512, 256, 1)
      self.conv3 = torch.nn.Conv1d(256, 128, 1)
      self.conv4 = torch.nn.Conv1d(128, category, 1)
      self.bn1 = nn.BatchNorm1d(512)
      self.bn2 = nn.BatchNorm1d(256)
      self.bn3 = nn.BatchNorm1d(128)
    self.relu = nn.ReLU()
    self.segmentation = segmentation
  
  def forward(self, x): # (batch, n_points, 3)
    batch, n_points, _ = x.shape
    x = x.transpose(1, 2)
    global_feature, local_feature, trans_mat = self.feat(x) # (b, 1024), (b, 64, n), (b, 64, 64)
    if self.segmentation is False:
      x = self.bn1(self.linear1(global_feature))
      x = self.bn2(self.dropout(self.linear2(x)))
      x = self.linear3(x)
      return x, trans_mat # (b, category), (b, 64, 64)
    else:
      concat_feature = torch.cat((local_feature, global_feature.unsqueeze(-1).repeat(1, 1, n_points)), dim=1) # (b, 1088, n)
      x = self.bn1(self.conv1(concat_feature))
      x = self.bn2(self.conv2(x))
      x = self.bn3(self.conv3(x))
      x = self.conv4(x) # (b, category, n)
      return x, trans_mat

class OrthogonalRegLoss(nn.Module):
  def __init__(self, alpha=1e-4):
    super(OrthogonalRegLoss, self).__init__()
    self.alpha = alpha
  
  def forward(self, mat):
    batch, dim, _ = mat.shape
    iden = torch.eye(dim).unsqueeze(0)
    iden = iden.cuda() if mat.is_cuda else iden
    return torch.mean(torch.norm(torch.bmm(mat, mat.transpose(1, 2)) - iden, dim=(1,2))) * self.alpha

# Training

In [13]:
dataset = ModelNetDataset(DATAPATH, mode='train')
dataloader = DataLoader(
  dataset,
  batch_size=BATCHSIZE,
  shuffle=True,
  num_workers=WORKERS
)

In [ ]:
classifier = PointNet(category=dataset.category, segmentation=False).to(device)
optimizer = optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
loss_fn = nn.CrossEntropyLoss()
orthloss = OrthogonalRegLoss(alpha=1e-4)
batch_num = len(dataloader)
for epoch in range(EPOCH):
  losses, accuracy = [], []
  for i, (points, category) in enumerate(dataloader):
    points, category = points.to(device), category.to(device)
    target = category.flatten()
    optimizer.zero_grad()
    classifier = classifier.train()
    pred, trans_mat = classifier(points)
    loss = loss_fn(pred, target) + orthloss(trans_mat) 
    loss.backward()
    losses.append(loss.item())
    optimizer.step()
    scheduler.step()
    pred_choice = pred.argmax(dim=1)
    correct = (pred_choice == target).sum()
    accuracy.append(correct.item() / BATCHSIZE)
    if i > 0 and ((i+1) % 50 == 0 or i+1 == batch_num):
      print('[Avg Epoch %d: %d/%d] train avg loss: %f accuracy: %f' % (epoch, i+1, batch_num, sum(losses)/len(losses), sum(accuracy)/len(accuracy)))
  torch.save(classifier.state_dict(), "modelnet.pt")

[Epoch 0: 10/246] train loss: 3.539261 accuracy: 0.062500
[Epoch 0: 20/246] train loss: 3.728072 accuracy: 0.062500
[Epoch 0: 30/246] train loss: 3.493363 accuracy: 0.125000
[Epoch 0: 40/246] train loss: 3.186347 accuracy: 0.281250
[Epoch 0: 50/246] train loss: 3.398359 accuracy: 0.156250
[Epoch 0: 60/246] train loss: 3.340137 accuracy: 0.125000
[Epoch 0: 70/246] train loss: 3.244162 accuracy: 0.156250
[Epoch 0: 80/246] train loss: 3.194694 accuracy: 0.093750
[Epoch 0: 90/246] train loss: 3.309104 accuracy: 0.156250
[Epoch 0: 100/246] train loss: 3.196022 accuracy: 0.218750
[Epoch 0: 110/246] train loss: 3.516742 accuracy: 0.062500
[Epoch 0: 120/246] train loss: 3.202292 accuracy: 0.093750
[Epoch 0: 130/246] train loss: 3.300503 accuracy: 0.218750
[Epoch 0: 140/246] train loss: 3.219541 accuracy: 0.281250
[Epoch 0: 150/246] train loss: 3.532183 accuracy: 0.093750
[Epoch 0: 160/246] train loss: 3.415588 accuracy: 0.156250
[Epoch 0: 170/246] train loss: 3.236504 accuracy: 0.187500
[Epoch